In [ ]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from dateutil import parser
from dateutil.relativedelta import relativedelta
from collections import Counter
import math

# Open and Pre-process

In [2]:
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location)

reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [3]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [4]:
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) 
pph['time'] = get_pph_date_strings(pph.time) 
# subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks 
# day 3: cycle not -1. day 2: cycle not -1. Day 1: cycle 6. Category: categorical. 
#outlooks = outlooks[(((outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6)) | ((outlooks['DAY'] == 2) & (outlooks['CYCLE'] != -1)) | ((outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1)))
#        & (outlooks['CATEGORY'] == 'CATEGORICAL')]

# reset incicies
outlooks = outlooks.reset_index(drop=True)
reports = reports.drop(columns=['geometry'])

# Gridize outlooks

In [91]:
def get_outlooks_subset(outlooks, outlook_type):
    outlooks = outlooks[(outlooks['THRESHOLD'] != 'SIGN') & (outlooks['THRESHOLD'] != 'TSTM')]
    if outlook_type == 'Day 3':
        return outlooks[(outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 7':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 17':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 1 Wind':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 1 Hail':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 1 Tornado':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'TORNADO')]
    
    elif outlook_type == 'Day 2 7 Wind':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 2 17 Wind':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 2 7 Hail':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 2 17 Hail':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 2 7 Tornado':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'TORNADO')]
    elif outlook_type == 'Day 2 17 Tornado':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'TORNADO')]
    
    raise Exception("Invalid outlook_type given to get_outlooks_subset")

In [63]:
# as in mca: create_gridded_outlook_dataset. But will need to work overall, not just by hazard. day 3 is overall probability, days 1 and 2 by hazard
# day 3 use ANY SEVERE, day 2 use ANY SEVERE, day 1 construct from each hazard (assuming independence? or use highest prob, which basically assumes complete dependence? Is this what is done for day 2?)
# Things are entirely categorical before 2002, so even more reason to make that the cutoff.

outlook_types = ['Day 3', 'Day 2 7', 'Day 2 17', 
                 'Day 2 7 Wind', 'Day 2 7 Hail', 'Day 2 7 Tornado', 
                 'Day 2 17 Wind', 'Day 2 17 Hail', 'Day 2 17 Tornado', 
                 'Day 1 Wind', 'Day 1 Hail', 'Day 1 Tornado', 
                 'Day 1']

outlook_dataset_new = xr.Dataset(
    data_vars=dict(
        lat=(['y', 'x'], pph['lat'].data),
        lon=(['y', 'x'], pph['lon'].data)
    ),
    coords=dict(
        time=(['time'], pph['time'].data),
        x=(['x'], pph['x'].data),
        y=(['y'], pph['y'].data),
        outlook=(['outlook'], outlook_types)
    ),
    attrs=dict(description="outlook as a percentage as a function of date, lat/lon, and which hazard type",
            grid = pph.grid),
)

outlook_dataset_new = outlook_dataset_new.assign(prob = (('time', 'y', 'x', 'outlook'), np.full((len(outlook_dataset['time']), len(outlook_dataset['y']), len(outlook_dataset['x']), len(outlook_types)), 0.0)))





In [7]:
# if picking up partway through
outlook_dataset = xr.open_dataset('data/outlooks/grid_outlooks2.nc')

In [99]:
# for each outlook type
for outlook_type in ['Day 2 7 Wind', 'Day 2 7 Hail', 'Day 2 7 Tornado', 'Day 2 17 Wind', 'Day 2 17 Hail', 'Day 2 17 Tornado']:
    print(outlook_type)
    outlooks_subset = get_outlooks_subset(outlooks, outlook_type)
    oldyear = 0
    array = np.zeros((len(outlook_dataset_new.time.values), len(outlook_dataset_new.y.values), len(outlook_dataset_new.x.values)))

    for date, i in zip(outlook_dataset_new.time.values, range(len(outlook_dataset_new.time.values))):
        lats = outlook_dataset_new.lat
        lons = outlook_dataset_new.lon
        datestring = date
        
        # printing to track progress
        year = datestring[0:4]
        if year != oldyear:
            print(year)
            oldyear = year

        outlooks_date = outlooks_subset[outlooks_subset['DATE'] == datestring]
        if (len(outlooks_date) > 0):
            for x, j in zip(outlook_dataset_new.x.values, range(len(outlook_dataset_new.x.values))):
                for y, k in zip(outlook_dataset_new.y.values, range(len(outlook_dataset_new.y.values))):
                    lat = lats.sel(x = x, y = y).values
                    lon = lons.sel(x = x, y = y).values
                    array[i, k, j] = find_threshold(outlooks_date, lat, lon)
    me = xr.DataArray(array, coords={'time': outlook_dataset_new.time.values, 'y': outlook_dataset_new.y.values, 'x': outlook_dataset_new.x.values},
                      dims=['time', 'y', 'x'])
    outlook_dataset_new['prob'].loc[dict(outlook = outlook_type)] = me
    outlook_dataset_new.to_netcdf('data/outlooks/grid_outlooks_day_2_hazards.nc')

Day 2 7 Wind
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [98]:
outlook_dataset_new['prob'].sel(outlook = 'Day 2 7 Wind').max()

<xarray.DataArray 'prob' ()>
array(0.)
Coordinates:
    outlook  <U16 'Day 2 7 Wind'

In [16]:
save_location = 'data/outlooks/grid_outlooks2.nc'
outlook_dataset.to_netcdf(save_location)

In [62]:
# build day 1 and Feb 1 2020- day 2 probabilities: max (assuming total dependence) seems right
for outlook in ['Day 2 7', 'Day 2 17', 'Day 1']:
    for time in outlook_dataset_new['time']:
        m = outlook_dataset_new['prob'].sel(time = time, outlook = [outlook + ' Wind', outlook + ' Hail', outlook + ' Tornado']).max(dim = outlook)
        outlook_dataset_new['prob'].loc[dict(time = time)] = m

<xarray.DataArray 'prob' ()>
array(0.)
Coordinates:
    time     <U12 '202004060000'
    outlook  <U7 'Day 2 7'

# Save files